Python version of test_SPDEDRIFT.cpp

In [ ]:
import gstlearn as gl
import matplotlib.pyplot as plt
import gstlearn.plot as gp
import os 

path = os.path.join(os.pardir,os.pardir,"data","Scotland")                  
verbose = False;
flagSPDE = True;
ndim = 2;

filename = os.path.join(path,"temperatures.ascii")
temperatures = gl.Db.createFromNF(filename,verbose);
temperatures.setLocator("January_temp", gl.ELoc.Z);

#temperatures.display();


filename = os.path.join(path,"grid.ascii")

grid = gl.DbGrid.createFromNF(filename,verbose);
#grid.display();

filename = os.path.join(path,"model.ascii")
model = gl.Model.createFromNF(filename,verbose);

#model.display();

filename = os.path.join(path,"vario.ascii")
vario = gl.Vario.createFromNF(filename,verbose);

#vario.display();

structs = [gl.ECov.NUGGET,gl.ECov.BESSEL_K];
consNug = gl.ConsItem.define(gl.EConsElem.SILL,0,0,0, gl.EConsType.UPPER,0.1);
consParam = gl.ConsItem.define(gl.EConsElem.PARAM,1, 0, 0, gl.EConsType.EQUAL,1.);
constraints = gl.Constraints()
constraints.addItem(consNug);
constraints.addItem(consParam);

opt = gl.Option_AutoFit()
gl.OptDbg.define(gl.EDbg.CONVERGE);
model.fit(vario,structs,False,opt,constraints);
#model.display();

spde = gl.SPDE()
spde.init(model,grid,temperatures,gl.ESPDECalcMode.KRIGING);
coeffs = spde.getCoeffs();
spde.compute();
spde.query(grid);

In [ ]:
ax = gp.grid(grid,usesel=True)

In [ ]:
neighU = gl.NeighUnique.create(ndim, False);
gl.kriging(temperatures, grid, model, neighU);

In [ ]:
plt.scatter(grid["spde*"],grid["*estim"],s=1)
plt.plot([-6,4],[-6,4])